# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = pd.read_csv("cities.csv") 
cities.head()

,Unnamed: 0,City,Cloudiness,Max Temp,Latitutde,Longitude,Humidity,Wind Speed,Country,Date
0,0,cherskiy,87,4.60,68.75,161.30,95,6.35,RU,39600
1,1,chokurdakh,100,15.48,70.63,147.92,97,6.85,RU,39600
2,2,albany,75,50.00,42.60,-73.97,33,12.75,US,-14400
3,3,hobart,90,51.80,-42.88,147.33,57,11.41,AU,36000
4,4,ushuaia,75,46.40,-54.80,-68.30,70,33.33,AR,-10800


In [3]:
print(len(cities))

582


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#configure gmaps
gmaps.configure(api_key=g_key)

#store locations
locations = cities[["Latitutde", "Longitude"]].astype(float)

#use humidity as the weight
humidityWeight = cities['Humidity']



In [5]:
# Heatmap layer

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidityWeight, dissipating=False, max_intensity=100, point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#Define Ideal state
maxTemp = 70
maxWind = 4
maxCloud_cover = 80
maxHumidity = 50

narrowed_city_df = cities.loc[(cities['Max Temp'] <= maxTemp) & (cities['Wind Speed'] <= maxWind) & (cities['Cloudiness'] <= maxCloud_cover) & (cities['Humidity'] <= maxHumidity)]

                                                       
                                                       
narrowed_city_df.dropna()                                                
narrowed_city_df.head()

,Unnamed: 0,City,Cloudiness,Max Temp,Latitutde,Longitude,Humidity,Wind Speed,Country,Date
140,140,panzhihua,0,65.08,26.59,101.71,27,2.35,CN,28800
197,197,mizan teferi,58,68.94,6.98,35.58,48,1.92,ET,10800
221,221,kitob,9,62.76,39.08,66.83,41,3.53,UZ,18000
234,234,saint george,1,68.00,37.10,-113.58,42,3.36,US,-21600
242,242,weiser,1,61.00,44.25,-116.97,38,3.36,US,-21600


In [7]:
narrowed_city_df['Lat/Lon'] = narrowed_city_df['Latitutde'].astype(str)+', ' + narrowed_city_df['Longitude'].astype(str)
narrowed_city_df.reset_index()
narrowed_city_df.head()


C:\Users\studi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Unnamed: 0,City,Cloudiness,Max Temp,Latitutde,Longitude,Humidity,Wind Speed,Country,Date,Lat/Lon
140,140,panzhihua,0,65.08,26.59,101.71,27,2.35,CN,28800,"26.59, 101.71"
197,197,mizan teferi,58,68.94,6.98,35.58,48,1.92,ET,10800,"6.98, 35.58"
221,221,kitob,9,62.76,39.08,66.83,41,3.53,UZ,18000,"39.08, 66.83"
234,234,saint george,1,68.00,37.10,-113.58,42,3.36,US,-21600,"37.1, -113.58"
242,242,weiser,1,61.00,44.25,-116.97,38,3.36,US,-21600,"44.25, -116.97"


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [40]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}
hotel_df = pd.DataFrame(columns=["Hotel Name", "lat", "lng"])
narrowed_city_df["Hotel Name"] = ""
x = 0
# use iterrows to iterate through pandas dataframe
for index, row in narrowed_city_df.iterrows():

    # get location type from df
    lat_lngs = row['Lat/Lon']
    # get city name
    citi_name = row["City"]

    # add keyword to params dict
    params['location'] = lat_lngs

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {citi_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        narrowed_city_df.loc[index, "Hotel Name"] = (results[0]['name'])
        hotel_df.loc[x, "Hotel Name"] = (results[0]['name'])
        hotel_df.loc[x, "lat"] = (results[0]['geometry']['location']['lat'])
        hotel_df.loc[x, "lng"] = (results[0]['geometry']['location']['lng'])
        

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        narrowed_city_df.loc[index, "Hotel Name"] = np.NaN
    print("------------")
    x = x + 1

C:\Users\studi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Retrieving Results for Index 140: panzhihua.
Closest hotel is Electric Power Hotel.
------------
Retrieving Results for Index 197: mizan teferi.
Closest hotel is Hotel Salayish | ሆቴል ሳላይሽ.
------------
Retrieving Results for Index 221: kitob.
Closest hotel is ULUG'BEK Hotel&Spa.
------------
Retrieving Results for Index 234: saint george.
Closest hotel is Best Western Plus Abbey Inn.
------------
Retrieving Results for Index 242: weiser.
Closest hotel is The Colonial Motel.
------------
Retrieving Results for Index 251: ust-kan.
Closest hotel is Mini-hotel na Leninskoy.
------------
Retrieving Results for Index 295: ponta do sol.
Closest hotel is Hotel do Campo.
------------
Retrieving Results for Index 391: teeli.
Missing field/result... skipping.
------------
Retrieving Results for Index 453: sabzevar.
Closest hotel is پارکینگ وسایل سنگین جنوب.
------------
Retrieving Results for Index 494: mugur-aksy.
Missing field/result... skipping.
------------


In [48]:
narrowed_city_df.dropna(inplace = True)
print (len(test))

8


C:\Users\studi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [49]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [50]:
print (f'{(len(locations))}, {len(narrowed_city_df)}')

8, 8


In [37]:
narrowed_city_df

,Unnamed: 0,City,Cloudiness,Max Temp,Latitutde,Longitude,Humidity,Wind Speed,Country,Date,Lat/Lon,Hotel Name
140,140,panzhihua,0,65.08,26.59,101.71,27,2.35,CN,28800,"26.59, 101.71",Electric Power Hotel
197,197,mizan teferi,58,68.94,6.98,35.58,48,1.92,ET,10800,"6.98, 35.58",Hotel Salayish | ሆቴል ሳላይሽ
221,221,kitob,9,62.76,39.08,66.83,41,3.53,UZ,18000,"39.08, 66.83",ULUG'BEK Hotel&Spa
234,234,saint george,1,68.00,37.10,-113.58,42,3.36,US,-21600,"37.1, -113.58",Best Western Plus Abbey Inn
242,242,weiser,1,61.00,44.25,-116.97,38,3.36,US,-21600,"44.25, -116.97",The Colonial Motel
251,251,ust-kan,0,42.17,50.93,84.76,43,0.69,RU,25200,"50.93, 84.76",Mini-hotel na Leninskoy
295,295,ponta do sol,8,64.99,32.67,-17.10,45,1.99,PT,3600,"32.67, -17.1",Hotel do Campo
391,391,teeli,24,45.97,51.03,90.23,34,2.46,RU,25200,"51.03, 90.23",
453,453,sabzevar,12,68.27,36.21,57.68,31,3.02,IR,16200,"36.21, 57.68",پارکینگ وسایل سنگین جنوب
494,494,mugur-aksy,31,36.19,50.35,90.50,33,2.93,RU,25200,"50.35, 90.5",


In [51]:
# Add marker layer ontop of heat map
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))